In [4]:
import pandas as pd
import os
import pandas_datareader.data as web
import pandas_datareader as pdr
from datetime import datetime
import matplotlib
import yfinance as yf
%matplotlib inline
matplotlib.style.use('ggplot') 

alpha_key = "EOMRLIZT4PTBH80M"

ntickers = pd.read_csv("nasdaq_ticker_info.txt", sep="\t")

# Hypothesis 1

### Firm Ratings actually influence future price

In [11]:
tickerSymbol = 'MSFT'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)
recs = tickerData.recommendations
recs.head()

,Firm,To Grade,From Grade,Action
Date,,,,
2012-03-16,Argus Research,Buy,,up
2012-03-19,Hilliard Lyons,Long-Term Buy,,main
2012-03-22,Morgan Stanley,Overweight,,main
2012-04-03,UBS,Buy,,main
2012-04-20,UBS,Buy,,main


In [18]:
actions = recs['Action'].drop_duplicates().values
print("Actions: ")
print(len(actions))
print(actions)
print("From Grades: ")
fromgrades = recs['From Grade'].drop_duplicates().values
print(len(fromgrades))
print(fromgrades)
print("To Grades: ")
togrades = recs['To Grade'].drop_duplicates().values
print(len(togrades))
print(togrades)
print("Firms: ")
firms = recs['Firm'].drop_duplicates().values
print(len(firms))
print(firms)

Actions: 
4
['up' 'main' 'init' 'down']
From Grades: 
16
['' 'Outperform' 'Hold' 'Buy' 'Overweight' 'Long-term Buy' 'Neutral'
 'Equal-Weight' 'Equal-weight' 'Market Perform' 'Underperform'
 'Sector Perform' 'Sell' 'Long-Term Buy' 'Underweight' 'Strong Buy']
To Grades: 
14
['Buy' 'Long-Term Buy' 'Overweight' 'Neutral' 'Hold' 'Sector Perform'
 'Equal-Weight' '' 'Market Perform' 'Sell' 'Outperform' 'Underperform'
 'Underweight' 'Strong Buy']
Firms: 
53
['Argus Research' 'Hilliard Lyons' 'Morgan Stanley' 'UBS' 'Bank oferica'
 'Goldman Sachs' 'Deutsche Bank' 'McAdams Wright Ragen'
 'Griffin Securities' 'Longbow Research' 'Jefferies' 'RBC Capital'
 'Barclays' 'Nomura' 'MKM Partners' 'Standpoint Research' 'BMO Capital'
 'SunTrust Robinson Humphrey' 'Argus' 'Bank of America' 'BGC Financial'
 'Credit Agricole' 'Stifel Nicolaus' 'Cowen & Co.' 'Atlantic Equities'
 'Evercore Partners' 'Citigroup' 'Credit Suisse' 'Sterne Agee'
 'B. Riley FBR' 'FBR Capital' 'JP Morgan' 'Oppenheimer'
 'Summit Researc

---

# Hypothesis 2

### ___________